In [ ]:
import pandas as pd
df = pd.read_csv('AAPL.csv', index_col='Date', parse_dates=True)
df.drop(['Open','Low','High','Adj Close','Volume'],axis=1,inplace=True)
df

In [ ]:
df = df.loc['2020-03-01':'2020-03-31'].copy()
df.head()


In [ ]:
# calculate Simple Moving Average with a window of 3 days
sma_window = 3
df['SMA'] = df['Close'].rolling(window = sma_window).mean()
df


In [ ]:
import matplotlib.pyplot as plt

# Plot the original data and SMA
plt.figure(figsize=(13, 6))
plt.plot(df['Close'], label='Original Close')
plt.plot(df['SMA'], label=f'SMA ({sma_window}-day)')
plt.title('Simple Moving Average for AAPL')
plt.xlabel('Date')
plt.ylabel('Value')
plt.legend()
plt.show()


In [ ]:
alpha = 0.3
df['EMA'] = df['Close'].ewm(alpha=alpha, adjust=False).mean()
df


In [ ]:
import matplotlib.pyplot as plt

# Plot the original data and EMA
plt.figure(figsize=(13, 6))
plt.plot(df['Close'], label='Original Close')
plt.plot(df['EMA'], label=f'EMA ({sma_window}-day)')
plt.title('Exponential Moving Average for AAPL')
plt.xlabel('Date')
plt.ylabel('Value')
plt.legend()
plt.show()


In [ ]:
span = 2
df['EMA'] = df['Close'].ewm(span=span, adjust=False).mean()
df


In [ ]:
df = pd.read_csv('AAPL.csv', index_col='Date', parse_dates=True)
df.drop(['Open','Low','High','Adj Close','Volume'],axis=1,inplace=True)
df


In [ ]:
df["Diff"] = df["Close"].diff()
df


In [ ]:
df = df.assign(Up=df["Diff"].apply(lambda x: x if x>=0 else 0))
df = df.assign(Down=df["Diff"].apply(lambda x: abs(x) if x<0 else 0))
df


In [ ]:
# average for the past 14 days
df = df.assign(RollUp=df["Up"].rolling(14).mean())        
df = df.assign(RollDown=df["Down"].rolling(14).mean())
df


In [ ]:
df = df.assign(RS = df["RollUp"]/df["RollDown"])
df = df.assign(RSI = 100 - (100/(1+df["RS"])))
df


In [ ]:
import matplotlib.pyplot as plt

fig, axes = plt.subplots(2,figsize=(12,10))
fig.suptitle('AAPL Price Movements and its RSI')
axes[0].plot(df.index, df['Close'])
axes[1].plot(df.index, df['RSI'], color='orange')

plt.axhline(y=30, color='r', linestyle='--')
plt.axhline(y=70, color='b', linestyle='--')


In [ ]:
import pandas as pd
def RSI(ohlc, period = 14, method = "SMA"):
    delta = ohlc["Close"].diff()
    up, down = delta.copy(), delta.copy()
    up[up < 0] = 0
    down[down > 0] = 0
       
    if method == "SMA":
        # using Smooth Moving Average
        gain = up.rolling(period).mean()
        loss = down.abs().rolling(period).mean()
    else:
        # using Exponential Weighted Moving Average
        gain = up.ewm(span=period).mean()
        loss = down.abs().ewm(span=period).mean()

    RS = gain / loss
    ohlc['RSI'] =  pd.Series(100 - (100 / (1 + RS)), name="RSI")
    
    return ohlc


In [ ]:
import time
from datetime import datetime, timedelta
import pandas as pd

# starts 365 days ago today
start_date = datetime.now() - timedelta(days=365)
start_date = int(round(start_date.timestamp()))

# ends today
end_date = datetime.now()
end_date = int(round(end_date.timestamp()))

# list of stocks
stocks = ['AAPL','GOOG','AMZN','NVDA']

# create an empty dataframe for storing the stocks
df_RSI = pd.DataFrame(columns=stocks)

# fetch the price of each stocks
for s in stocks:
    df = pd.read_csv(f"https://query1.finance.yahoo.com/v7/finance/download/{s}?period1={start_date}&period2={end_date}&interval=1d&events=history&includeAdjustedClose=true")  
    # get the RSI of each stock
    df = RSI(df, method='SMA')
    
    # add the RSI to the df
    df_RSI[s] = df['RSI']
    
df_RSI.index = df['Date']       
display(df_RSI)


In [ ]:
# !pip install plotly
# !pip install cufflinks


In [ ]:
from plotly.offline import download_plotlyjs, init_notebook_mode, \
     plot, iplot
import cufflinks as cf

init_notebook_mode(connected=True)
cf.go_offline()  

fig = df_RSI.iplot(asFigure=True)  # uses Cufflinks to plot from dataframe
fig


In [ ]:
fig.add_shape(dict(
    type='line',
    y0=30,
    y1=30,
    x0=df_RSI.index[0],
    x1=df_RSI.index[-1],
    line=dict(color='red', width=1, dash='dot'),
))

fig.add_shape(dict(
    type='line',
    y0=70,
    y1=70,
    x0=df_RSI.index[0],
    x1=df_RSI.index[-1],
    line=dict(color='blue', width=1, dash='dot'),
))


In [ ]:
import pandas as pd
df = pd.read_csv("AAPL.csv", index_col="Date", parse_dates=True)


In [ ]:
import numpy as np
np.corrcoef(np.arange(len(df)), 
            df['Open'])


In [ ]:
np.corrcoef(df['Open'][:-1].values,
            df['Open'][1:].values)[0,1]


In [ ]:
autocorrelation = []
for shift in range(1,25):
    correlation = np.corrcoef(df['Open'][:-shift].values,
                              df['Open'][shift:].values)[0,1]
    autocorrelation.append(correlation)   


In [ ]:
plt.stem(autocorrelation)

In [ ]:
# pip install statsmodel
from statsmodels.graphics.tsaplots import plot_acf
plot_acf(df['Open'])


In [ ]:
from statsmodels.graphics.tsaplots import plot_pacf
plot_pacf(df['Open'])


In [ ]:
df['Diff'] = df['Open'].diff()
df


In [ ]:
df[['Open','Diff']].plot(subplots=True, layout=(2,1))

In [ ]:
from statsmodels.graphics.tsaplots import plot_acf
import matplotlib.pyplot as plt

fig, (ax1, ax2) = plt.subplots(2)
plot_acf(df['Open'], ax=ax1)
plot_acf(df['Diff'], ax=ax2)


In [ ]:
from statsmodels.graphics.tsaplots import plot_pacf
_ = plot_pacf(df['Diff'].dropna())


In [ ]:
from statsmodels.graphics.tsaplots import plot_pacf
_ = plot_acf(df['Diff'].dropna())


In [ ]:
!pip install pmdarima

In [ ]:
from pmdarima.arima import auto_arima
model = auto_arima(df['Open'], 
                   test='adf',
                   seasonal=False,   
                   trace=True,
                   error_action='ignore',  
                   suppress_warnings=True, 
                   stepwise=True)


In [ ]:
train_df, test_df = df[0:int(len(df)*0.7)], df[int(len(df)*0.7):]

# history price from train set
x = [x for x in train_df['Open']]

# actual stock price from test set
y = test_df['Open']


In [ ]:
from statsmodels.tsa.arima.model import ARIMA
# used for storing predicted price for test set
predictions = list()

# rolling forecasts for the test data
for i in range(0, len(y)):
    model = ARIMA(x, order=(0,1,0))    
    model_fit = model.fit()
    
    # forecast the next stock price
    forecasted_price = model_fit.forecast()[0]

    # append predicted price  
    predictions.append(forecasted_price)

    # get the actual price from test set and add it to x
    x.append(y.iloc[i])


In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(16,8))

# plot the stock price for the train set
plt.plot(train_df.index, 
         train_df['Open'], 
         color='green', 
         label = 'Historical AAPL Stock Price')

# actual stock price for the test set
plt.plot(test_df.index, 
         y, 
         color = 'red', 
         label = 'Actual AAPL Stock Price')

# predicted stock price for the test set
plt.plot(test_df.index, 
         predictions, 
         color = 'blue', 
         label = 'Predicted AAPL Stock Price')

plt.title('Apple Stock Price Prediction')
plt.xlabel('Date')
plt.ylabel('Price')
plt.legend()
plt.grid(True)
plt.savefig('AAPL_arima_model.pdf')
plt.show()


In [ ]:
model = ARIMA(x, order=(0,1,0))
model_fit = model.fit()
model_fit.forecast()


In [ ]:
import pandas as pd
import numpy as np
date_range = pd.date_range(start = "2023-06-01", 
                           end   = "2023-06-30 23:59:00",
                           freq  = "H")


In [ ]:
ts = pd.Series(list(np.random.randn(len(date_range))),
               index = date_range)


In [ ]:
import matplotlib.pyplot as plt
import seaborn
fig, ax = plt.subplots(figsize=(20,5))
seaborn.boxplot(x = ts.index.dayofyear, 
                y = ts, 
                ax = ax)


In [ ]:
import matplotlib.pyplot as plt
import seaborn
fig, ax = plt.subplots(figsize=(20,5))
seaborn.boxplot(x = ts.index.dayofyear, 
                y = ts, 
                ax = ax,
                hue = ts.index.dayofyear,
                palette = 'Spectral',
                legend = False)


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn
date_range = pd.date_range(start = "2023-06-01", 
                           end   = "2023-06-30 23:59:00",
                           freq  = "H")
df = pd.DataFrame(
    {
        'temp':np.random.randn(len(date_range))
    }, index = date_range)
df


In [ ]:
fig, ax = plt.subplots(figsize=(20,5))
seaborn.boxplot(x = df.index.dayofyear, 
                y = df['temp'], 
                ax = ax,
                hue = ts.index.dayofyear,
                palette = 'Spectral',
                legend = False)


In [ ]:
df = pd.read_csv('WetBulbTemperatureHourly.csv', 
                 parse_dates = ['wbt_date'], 
                 index_col='wbt_date')
df


In [ ]:
fig, ax = plt.subplots(figsize=(12,5))
seaborn.boxplot(x = df.index.month,
                y = df['wet_bulb_temperature'], 
                ax = ax,
                hue = df.index.month,
                palette = 'Spectral',
                legend = False)


In [ ]:
fig, ax = plt.subplots(figsize=(24,10))
seaborn.boxplot(x = df.index.year,
                y = df['wet_bulb_temperature'], 
                ax = ax,
                hue = df.index.year,
                palette = 'Spectral',
                legend = False)


In [ ]:
fig, ax = plt.subplots(figsize=(24,10))
seaborn.boxplot(x = df.index.year,
                y = df['wet_bulb_temperature'], 
                ax = ax,
                hue = df.index.year,
                palette = 'Spectral',
                legend = False)
ax.set_xticks(ax.get_xticks())
_ = ax.set_xticklabels(ax.get_xticklabels(), rotation=30)


In [ ]:
fig, ax = plt.subplots(figsize=(24,10))
seaborn.boxplot(
    x = df['1998-01-01':'1998-12-31'].index.month,
    y = df['1998-01-01':'1998-12-31']['wet_bulb_temperature'], 
    ax = ax,
    hue = df['1998-01-01':'1998-12-31'].index.month,
    palette = 'Spectral',
    legend = False)


In [ ]:
fig, ax = plt.subplots(figsize=(24,10))

seaborn.boxplot(
    x = df['1998-05-01':'1998-05-31'].index.day,
    y = df['1998-05-01':'1998-05-31']['wet_bulb_temperature'], 
    ax = ax,
    hue = df['1998-05-01':'1998-05-31'].index.day,
    palette = 'Spectral',
    legend = False)


In [ ]:
fig, ax = plt.subplots(figsize=(150,10))
seaborn.boxplot(
    x = df['1998-01-01':'1998-12-31'].index.dayofyear,
    y = df['1998-01-01':'1998-12-31']['wet_bulb_temperature'], 
    ax = ax,
    hue = df['1998-01-01':'1998-12-31'].index.dayofyear,
    palette = 'Spectral',
    legend = False)
fig.savefig('temp.jpg')


In [ ]:
fig, ax = plt.subplots(figsize=(150,10))
seaborn.boxplot(
    x = df['1998-01-01':'1998-12-31'].index.dayofyear,
    y = df['1998-01-01':'1998-12-31']['wet_bulb_temperature'], 
    ax = ax,
    hue = df['1998-01-01':'1998-12-31'].index.dayofyear,
    palette = 'Spectral',
    legend = False)

ax.set_xticks(ax.get_xticks())
ax.set_xticklabels(labels = 
    df['1998-01-01':'1998-12-31'].index.strftime(
        '%Y-%m-%d').sort_values().unique(), 
    rotation=45, ha='right')

fig.savefig('temp.jpg')
